In [1]:
# BPSK (Binary Phase Shift Keying) 통신 시스템 시뮬레이션

import numpy as np
from numpy import sqrt
from numpy.random import rand, randn
import matplotlib.pyplot as plt

# 채널 개수
n_channel = 8
# 데이터 비트 수
k = 8
# 코드 레이트
R = k / n_channel
# 심볼 수
M = 2 ** k

# 시뮬레이션 횟수
N = 50000
# Eb/N0 범위
EbNodB_range = range(0, 11)
# 8비트 BPSK의 비트 에러율을 저장할 리스트
BPSK_ber_8 = [None] * len(EbNodB_range)

# 무작위 데이터 생성
data_10 = np.random.randint(M, size=N)

# Eb/N0 값에 따른 에러율 계산
for n in range(0, len(EbNodB_range)):
    EbNodB = EbNodB_range[n]
    EbNo = 10.0 ** (EbNodB / 10.0)
    noise_std = 1 / sqrt(2 * EbNo)
    errors = 0

    for i in range(0, len(data_10)):
        data_2 = []
        temp = data_10[i]
        # 10진수 데이터를 2진수로 변환
        for j in range(0, k):
            data_2.append(temp % 2)
            temp = temp // 2
        data_2.reverse()

        # BPSK 모듈레이션과 노이즈 추가
        for j in range(0, len(data_2)):
            x = 2 * (data_2[j] >= 0.5) - 1
            y = x + noise_std * randn(1)
            y_d = 2 * (y >= 0) - 1
            # 수신 신호와 비트 에러 체크
            if (x != y_d):
                errors = errors + 1
                break

    # 비트 에러율 계산
    BPSK_ber_8[n] = 1.0 * errors / N
    # 결과 출력
    print("EbNodB:", EbNodB)
    print("Error bits:", errors)
    print("Error probability:", BPSK_ber_8[n])


EbNodB: 0
Error bits: 450
Error probability: 0.45
EbNodB: 1
Error bits: 345
Error probability: 0.345
EbNodB: 2
Error bits: 272
Error probability: 0.272
EbNodB: 3
Error bits: 165
Error probability: 0.165
EbNodB: 4
Error bits: 94
Error probability: 0.094
EbNodB: 5
Error bits: 51
Error probability: 0.051
EbNodB: 6
Error bits: 15
Error probability: 0.015
EbNodB: 7
Error bits: 10
Error probability: 0.01
EbNodB: 8
Error bits: 1
Error probability: 0.001
EbNodB: 9
Error bits: 0
Error probability: 0.0
EbNodB: 10
Error bits: 1
Error probability: 0.001


In [1]:
# Functional API를 사용한 오토인코더 및 디지털 통신 시스템 시뮬레이션

# 라이브러리 임포트
import numpy as np
import tensorflow as tf
from keras.layers import Input, Dense, GaussianNoise, Lambda, BatchNormalization
from keras.models import Model
from keras.optimizers import Adam, SGD
from keras import backend as K

# 채널 및 데이터 설정
n_channel = 8
k = 8
R = k / n_channel
M = 2 ** k
print('M:', M, 'k:', k, 'n:', n_channel)

# 데이터 생성
N = 50000
label = np.random.randint(M, size=N)
data = tf.one_hot(label, M)
data = np.array(data)

# 오토인코더 정의
input_signal = Input(shape=(M,))
encoded = Dense(M, activation='relu')(input_signal)
encoded1 = Dense(n_channel, activation='linear')(encoded)
encoded2 = Lambda(lambda x: np.sqrt(n_channel)*K.l2_normalize(x,axis=1))(encoded1)
EbNo_train = 10**(0.7)  # 5.01187 converted 7 db of EbNo
encoded3 = GaussianNoise(np.sqrt(1/(2*R*EbNo_train)))(encoded2)
decoded = Dense(M, activation='relu')(encoded3)
decoded1 = Dense(M, activation='softmax')(decoded)
autoencoder = Model(input_signal, decoded1)
autoencoder.compile(optimizer='adam', loss='categorical_crossentropy')

# 오토인코더 훈련
N_val = 1000
val_label = np.random.randint(M, size=N_val)
val_data = tf.one_hot(val_label, M)
val_data = np.array(val_data)
autoencoder.fit(data, data, epochs=30, batch_size=100, validation_data=(val_data, val_data))

# 인코더와 디코더 생성
encoder = Model(input_signal, encoded2)
encoded_input = Input(shape=(n_channel,))
deco = autoencoder.layers[-2](encoded_input)
deco = autoencoder.layers[-1](deco)
decoder = Model(encoded_input, deco)

# BER 테스트 데이터 생성
N = 50000000
test_label = np.random.randint(M, size=N)
test_data = tf.one_hot(test_label, M)
test_data = np.array(test_data)

# 산점도 데이터 생성
scatter_plot = []
for i in range(0, M):
    temp = np.zeros(M)
    temp[i] = 1
    scatter_plot.append(encoder.predict(np.expand_dims(temp,axis=0)))
scatter_plot = np.array(scatter_plot)

# 산점도 그리기
import matplotlib.pyplot as plt
scatter_plot = scatter_plot.reshape(M, n_channel, 1)
plt.scatter(scatter_plot[:,0], scatter_plot[:,1])
plt.axis((-2.5, 2.5, -2.5, 2.5))
plt.grid()
plt.show()

# BER 계산
EbNodB_range = list(np.arange(0,11,2))
ber_8_F = [None]*len(EbNodB_range)
for n in range(0, len(EbNodB_range)):
    EbNo = 10.0**(EbNodB_range[n] / 10.0)
    noise_std = np.sqrt(1 / (2 * R * EbNo))
    noise_mean = 0
    no_errors = 0
    nn = N
    noise = noise_std * np.random.randn(nn, n_channel)
    encoded_signal = encoder.predict(test_data)
    final_signal = encoded_signal + noise
    pred_final_signal = decoder.predict(final_signal)
    pred_output = np.argmax(pred_final_signal, axis=1)
    no_errors = (pred_output != test_label)
    no_errors = no_errors.astype(int).sum()
    ber_8_F[n] = no_errors / nn
    print('SNR:', EbNodB_range[n], 'BER:', ber_8_F[n])


M: 256 k: 8 n: 8
Epoch 1/30
10/10 [==============================] - 11s 71ms/step - loss: 5.4683 - val_loss: 5.3238
Epoch 2/30
10/10 [==============================] - 0s 20ms/step - loss: 5.1532 - val_loss: 5.0823
Epoch 3/30
10/10 [==============================] - 0s 18ms/step - loss: 4.8772 - val_loss: 4.8382
Epoch 4/30
10/10 [==============================] - 0s 45ms/step - loss: 4.5905 - val_loss: 4.5756
Epoch 5/30
10/10 [==============================] - 0s 27ms/step - loss: 4.2813 - val_loss: 4.2962
Epoch 6/30
10/10 [==============================] - 0s 23ms/step - loss: 3.9471 - val_loss: 3.9969
Epoch 7/30
10/10 [==============================] - 0s 22ms/step - loss: 3.6062 - val_loss: 3.6939
Epoch 8/30
10/10 [==============================] - 0s 33ms/step - loss: 3.2611 - val_loss: 3.3887
Epoch 9/30
10/10 [==============================] - 0s 18ms/step - loss: 2.9254 - val_loss: 3.0904
Epoch 10/30
10/10 [==============================] - 0s 21ms/step - loss: 2.6246 - val_loss

ResourceExhaustedError: {{function_node __wrapped__OneHot_device_/job:localhost/replica:0/task:0/device:GPU:0}} OOM when allocating tensor with shape[50000000,256] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc [Op:OneHot] name: 

In [ ]:
# Sequential API를 사용한 오토인코더

import tensorflow as tf
import numpy as np
from keras.models import Sequential
from keras.layers import Input, Dense, GaussianNoise, Lambda, InputLayer, BatchNormalization
import matplotlib.pyplot as plt
from keras import backend as K

# 채널 및 데이터 크기 설정
n_channel = 8
k = 8
R = k/n_channel
M = 2**k
EbNo_train = 10**(0.7)

N = 50000
label = np.random.randint(M, size=N)

# 원핫 인코딩
data = tf.one_hot(label, M)
data = np.array(data)

# Sequential 모델 생성
autoencoder = Sequential()

# 입력 레이어 추가
autoencoder.add(InputLayer(input_shape=(M,)))

# 인코더 레이어 추가
autoencoder.add(Dense(M, activation='relu'))
autoencoder.add(Dense(n_channel, activation='linear'))

# 정규화 추가
autoencoder.add(Lambda(lambda x: np.sqrt(n_channel)*K.l2_normalize(x,axis=1)))

# 채널 추가
autoencoder.add(GaussianNoise(np.sqrt(1/(2*R*EbNo_train))))

# 디코더 레이어 추가
autoencoder.add(Dense(M, activation='relu'))
autoencoder.add(Dense(M, activation='softmax'))

# 모델 요약
autoencoder.summary()

# 컴파일
autoencoder.compile(optimizer='adam', loss='categorical_crossentropy')

# 검증을 위한 데이터 생성
N_val = 1000
val_label = np.random.randint(M, size=N_val)

val_data = tf.one_hot(val_label, M)
val_data = np.array(val_data)

# 학습
autoencoder.fit(data, data, epochs=100, batch_size=500, validation_data=(val_data, val_data))

# 테스트
encoder = Sequential()
encoder.add(InputLayer(input_shape=(M,)))
encoder.add(autoencoder.layers[0])
encoder.add(autoencoder.layers[1])
encoder.add(autoencoder.layers[2])
encoder.summary()

decoder = Sequential()
decoder.add(InputLayer(input_shape=(n_channel,)))
decoder.add(autoencoder.layers[-2])
decoder.add(autoencoder.layers[-1])
decoder.summary()

# BER 확인을 위한 데이터 생성
N = 100000
test_label = np.random.randint(M, size=N)

test_data = tf.one_hot(test_label, M)
test_data = np.array(test_data)

# 산점도 생성
scatter_plot = []
for i in range(0, M):
    temp = np.zeros(M)
    temp[i] = 1
    scatter_plot.append(encoder.predict(np.expand_dims(temp, axis=0)))
scatter_plot = np.array(scatter_plot)
print(scatter_plot.shape)

# 산점도 플로팅
scatter_plot = scatter_plot.reshape(M, n_channel, 1)
plt.scatter(scatter_plot[:,0], scatter_plot[:,1])
plt.axis((-2.5, 2.5, -2.5, 2.5))
plt.grid()
plt.show()

# BER 확인
EbNodB_range = list(np.arange(0, 11, 2))
ber_8_S = [None]*len(EbNodB_range)
for n in range(0, len(EbNodB_range)):
    EbNo = 10.0**(EbNodB_range[n]/10.0)
    noise_std = np.sqrt(1/(2*R*EbNo))
    noise_mean = 0
    no_errors = 0
    nn = N
    noise = noise_std * np.random.randn(nn, n_channel)
    encoded_signal = encoder.predict(test_data)
    final_signal = encoded_signal + noise
    pred_final_signal = decoder.predict(final_signal)
    pred_output = np.argmax(pred_final_signal, axis=1)
    no_errors = (pred_output != test_label)
    no_errors = no_errors.astype(int).sum()
    ber_8_S[n] = no_errors / nn
    print('SNR:', EbNodB_range[n], 'BER:', ber_8_S[n])


In [ ]:
# SNR 그래프

# ber 곡선 그리기
import matplotlib.pyplot as plt

plt.plot(EbNodB_range, ber_2_F, 'b^', label='Autoencoder_F (2,2)')
plt.plot(EbNodB_range, ber_8_F, 'go', label='Autoencoder_F (8,8)')
plt.plot(EbNodB_range, ber_2_S, 'y-', label='Autoencoder_S (2,2)')
plt.plot(EbNodB_range, ber_8_S, 'c-', label='Autoencoder_S (8,8)')
plt.plot(EbNodB_range, BPSK_ber_2, 'r:', label='BPSK (2,2)')
plt.plot(EbNodB_range, BPSK_ber_8, 'k:', label='BPSK (8,8)')

plt.yscale('log')
plt.xlabel('SNR 범위')
plt.ylabel('블록 에러율')
plt.grid()
plt.legend(loc='upper right', ncol=1)

# plt.savefig('AutoEncoder_2_2_constrained_BER_matplotlib')
plt.show()
